In [1]:
from matplotlib import pyplot
%matplotlib inline
import numpy as np
import pandas as pd
import os
import yt
import ytree

In [2]:
home_dir = '/home/jovyan/work/home'
data_dir = '/home/jovyan/work/data'

In [3]:
ds = yt.load(os.path.join(data_dir, 'DD0305/output_0305'))

yt : [INFO     ] 2021-02-16 00:35:47,337 Parameters: current_time              = 13.000427048413
yt : [INFO     ] 2021-02-16 00:35:47,338 Parameters: domain_dimensions         = [512 512 512]
yt : [INFO     ] 2021-02-16 00:35:47,339 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2021-02-16 00:35:47,340 Parameters: domain_right_edge         = [1. 1. 1.]
yt : [INFO     ] 2021-02-16 00:35:47,341 Parameters: cosmological_simulation   = 1
yt : [INFO     ] 2021-02-16 00:35:47,341 Parameters: current_redshift          = 14.799255636334
yt : [INFO     ] 2021-02-16 00:35:47,342 Parameters: omega_lambda              = 0.734
yt : [INFO     ] 2021-02-16 00:35:47,342 Parameters: omega_matter              = 0.266
yt : [INFO     ] 2021-02-16 00:35:47,343 Parameters: omega_radiation           = 0.0
yt : [INFO     ] 2021-02-16 00:35:47,344 Parameters: hubble_constant           = 0.71


In [4]:
halos = yt.load(os.path.join(data_dir, 'rockstar_halos', 'halos_DD0305.0.bin'))

yt : [INFO     ] 2021-02-16 00:36:04,339 Parameters: current_time              = 8944476102963221.0 s
yt : [INFO     ] 2021-02-16 00:36:04,340 Parameters: domain_dimensions         = [1 1 1]
yt : [INFO     ] 2021-02-16 00:36:04,341 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2021-02-16 00:36:04,342 Parameters: domain_right_edge         = [28.39999962 28.39999962 28.39999962]
yt : [INFO     ] 2021-02-16 00:36:04,343 Parameters: cosmological_simulation   = 1
yt : [INFO     ] 2021-02-16 00:36:04,344 Parameters: current_redshift          = 14.79925588506347
yt : [INFO     ] 2021-02-16 00:36:04,344 Parameters: omega_lambda              = 0.734000027179718
yt : [INFO     ] 2021-02-16 00:36:04,345 Parameters: omega_matter              = 0.26600000262260437
yt : [INFO     ] 2021-02-16 00:36:04,346 Parameters: omega_radiation           = 0.0
yt : [INFO     ] 2021-02-16 00:36:04,346 Parameters: hubble_constant           = 0.7099999785423279


In [5]:
ind = np.argsort(-halos.r[:]["particle_mass"])
x, y, z = halos.r[:]["particle_position"].T[:, ind].in_units("unitary").d
radius = halos.r[:]["virial_radius"][ind].in_units("kpc").d
mass = halos.r[:]["particle_mass"][ind].in_units("Msun")

yt : [INFO     ] 2021-02-16 00:36:07,315 Allocating for 5.051e+04 particles
Initializing refined index:  88%|████████▊ | 7/8 [00:17<00:02,  2.50s/it]


In [6]:
#make projections
for i, (hx, hy, hz, hr, m) in enumerate(zip(x, y, z, radius, mass[:3])):
    print(i, m)
    data_source = ds.r[hx-hr:hx+hr, hy-hr:hy+hr, hz-hr:hz+hr]
    print(data_source.min("density"))
    print(data_source.min("temperature"))
    p = yt.ProjectionPlot(ds, "x", ["density", "temperature"], weight_field="density", data_source=data_source)
    p.set_center((hy, hz))
    p.set_width(hr * 1.9)
    p.set_log("density", True)
    p.set_axes_unit("kpc")
    p.set_cmap("temperature", "hot")
    p.annotate_title('mass = {:.3e} Msun'.format(m))
    p.save("projection_%04i.png" % i)
    
    #p2 = yt.PhasePlot(data_source, "density", "temperature", ["cell_mass"], weight_field=None)
    #p2.save("phaseplot_%04i.png" % i)

0 1524047730.5669248 Msun


Parsing Hierarchy : 100%|█████████▉| 213949/213950 [04:00<00:00, 887.80it/s] 
yt : [INFO     ] 2021-02-16 00:41:54,579 Gathering a field list (this may take a moment.)


KeyboardInterrupt: 

In [ ]:
#make phase plots
for i, (hx, hy, hz, hr, m) in enumerate(zip(x, y, z, radius, mass[:3])):
    print(i, m)
    data_source = ds.r[hx-hr:hx+hr, hy-hr:hy+hr, hz-hr:hz+hr]
    #print(data_source.min("density"))
    #print(data_source.min("temperature"))
    data_source.profile(["density", "temperature"], "cell_mass", weight_field=None).plot().save("phaseplot_%04i.png" % i)
    #p = yt.PhasePlot(data_source, "density", "temperature", ["cell_mass"], weight_field=None)
    #p.save("phaseplot_%04i.png" % i)

In [ ]:
profile_specifications = [
   (["density", "temperature"], "cell_mass", None, "density_temperature_mass"),
   (["density", "temperature"], "El_fraction", "cell_mass", "density_temperature_Elfraction"),
   (["density", "velocity_magnitude"], "cell_mass", None, "density_vel_mass")
]

In [ ]:
prof = []

for i, (hx, hy, hz, hr, m) in enumerate(zip(x, y, z, radius, mass[:10])):
    print(i, m)
    data_source = ds.r[hx-hr:hx+hr, hy-hr:hy+hr, hz-hr:hz+hr]
    print(data_source.min("density"))
    print(data_source.min("temperature"))
    h = []
    for bins, color, weight, filename_prefix in profile_specifications:
        # if os.path.exists("halo_%04i/%s_%04i.png" % (i, filename_prefix, i)): continue
        p = data_source.profile(bins, color, weight_field=weight)
        p.plot().save("halo_%04i/%s_%04i.png" % (i, filename_prefix, i))
        h.append(p['cell_mass'], p.x_bins, p.y_bins)
    prof.append(h)

In [ ]:
import h5py

In [ ]:
hf = h5py.File('halos.hdf5', 'w')

In [ ]:
for i in range(10):
    g = hf.create_group('halo_%04i' % i)
    for h in prof:
        g1 = g.create_group('density_temperature_mass')
        g1.create_dataset('cell_mass', data = h[0])
        g1.create_dataset('density', data = h[1])
        g1.create_dataset('temperature', data = h[2])
        g1.flush()
        
        g2 = g.create_group('density_temperature_Elfraction')
        g2.create_dataset('El_fraction', data = h[3])
        g2.create_dataset('density', data = h[4])
        g2.create_dataset('temperature', data = h[5])
        g2.flush()
        
        g3 = g.create_group('density_vel_mass')
        g3.create_dataset('cell_mass', data = h[6])
        g3.create_dataset('density', data = h[7])
        g3.create_dataset('velocity_magnitude', h[8])
        g3.flush()
        